In [1]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler matplotlib_venn goatools gseapy scperturb biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 41.0.7 which is incompatible.


In [2]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns
import scvelo as scv
scv.settings.verbosity=1

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../')
from utils import *

# scperturb package
sys.path.insert(1, '../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../figures/')

In [3]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')

In [5]:
sorted([file.name for file in (TEMPDIR / 'DraegerKampmann2022/').glob('*')])

['GSM5387652_iTF_Microglia_10X_Lane1_filtered_feature_bc_matrix.h5',
 'GSM5387652_iTF_Microglia_10X_Lane1_raw_feature_bc_matrix.h5',
 'GSM5387653_iTF_Microglia_10X_Lane2_filtered_feature_bc_matrix.h5',
 'GSM5387653_iTF_Microglia_10X_Lane2_raw_feature_bc_matrix.h5',
 'GSM5387654_iTF_Microglia_10X_Lane3_filtered_feature_bc_matrix.h5',
 'GSM5387654_iTF_Microglia_10X_Lane3_raw_feature_bc_matrix.h5',
 'GSM5387655_iTF_Microglia_10X_Lane4_filtered_feature_bc_matrix.h5',
 'GSM5387655_iTF_Microglia_10X_Lane4_raw_feature_bc_matrix.h5',
 'GSM5387656_iTF_Microglia_sgRNAenrichment_Lane1_filtered_feature_bc_matrix.h5',
 'GSM5387656_iTF_Microglia_sgRNAenrichment_Lane1_raw_feature_bc_matrix.h5',
 'GSM5387657_iTF_Microglia_sgRNAenrichment_Lane2_filtered_feature_bc_matrix.h5',
 'GSM5387657_iTF_Microglia_sgRNAenrichment_Lane2_raw_feature_bc_matrix.h5',
 'GSM5387658_iTF_Microglia_sgRNAenrichment_Lane3_filtered_feature_bc_matrix.h5',
 'GSM5387658_iTF_Microglia_sgRNAenrichment_Lane3_raw_feature_bc_matrix.h5

In [ ]:
adata = sc.read_10x_h5(TEMPDIR / 'DraegerKampmann2022/GSM5387652_iTF_Microglia_10X_Lane1_filtered_feature_bc_matrix.h5', gex_only=False)

In [26]:
adata.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC233755.2,ENSG00000277856,Gene Expression,GRCh38
AC233755.1,ENSG00000275063,Gene Expression,GRCh38
AC240274.1,ENSG00000271254,Gene Expression,GRCh38
AC213203.1,ENSG00000277475,Gene Expression,GRCh38


In [41]:
adata.var.value_counts('feature_types')

feature_types
Gene Expression    33538
dtype: int64

In [29]:
sdata = sc.read_10x_h5(TEMPDIR / 'DraegerKampmann2022/GSM5387656_iTF_Microglia_sgRNAenrichment_Lane1_filtered_feature_bc_matrix.h5', gex_only=False)

In [30]:
sdata

AnnData object with n_obs × n_vars = 59551 × 33538
    var: 'gene_ids', 'feature_types', 'genome'

In [38]:
sdata.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC233755.2,ENSG00000277856,Gene Expression,GRCh38
AC233755.1,ENSG00000275063,Gene Expression,GRCh38
AC240274.1,ENSG00000271254,Gene Expression,GRCh38
AC213203.1,ENSG00000277475,Gene Expression,GRCh38


In [40]:
sdata.var.value_counts('feature_types')

feature_types
Gene Expression    33538
dtype: int64

In [44]:
sdata.var_names[sdata.var.index.str.startswith('AARS')]

Index(['AARS2', 'AARS', 'AARSD1'], dtype='object')

In [45]:
sdata.var_names[sdata.var.index.str.startswith('CDK12')]

Index(['CDK12'], dtype='object')

In [ ]:
# There is no sgRNA-seq info in here. I wrote to the original authors (01.12.2023)